# 23 Day 05

https://adventofcode.com/2023/day/5


In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=5)

def test(method, input, expected):
    actual = method(*input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')


Let's make sure that the example contains all the actual map value

In [ ]:
puzzle.input_data

import re
re.findall(r'\w+-to-\w+', puzzle.input_data)

Ok, it matches the example. Whew!

In [ ]:
from collections import defaultdict


class Almanac:

    def __init__(self, input_data:str):
        input_blocks = input_data.split('\n\n')
        
        self.seeds = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        
        maps = {}
        # Do I need this?
        # maps = {('seed', 'seed'): seeds}

        # I could not get this to go in one block. I'll ask for some help here.
        # (\w+)-to-(\w+) map:\n(:?(\d+) (\d+) (\d+)\n?)+
        # https://regex101.com/r/EtIaGl/1
        # Deletion Link: https://regex101.com/delete/pRW1eIlXymmnzhMO4O44tLemCOriUzaGGgkA
        for block in input_blocks[1:]:
            # print(block)
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()

            mapping = {}
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                dest_start, source_start, range_len = m
                for src, dest in zip(range(int(source_start), int(source_start) + int(range_len)), range(int(dest_start) , int(dest_start) + int(range_len))):
                    # TODO Validate this isn't already populated in case there are a list of options
                    mapping[src] = dest
            maps[map_label] = mapping
            
        self.maps = maps
            # TODO: There must be a better way to accomplish this in Pandas...
            # for indx, value in df[map_label[0]].items():
            #     # print(df[indx])
            #     print(df.iloc[indx])
            # row = df.loc[df[map_label[1]] == some_value]
            # df[map_label[1]] = df.apply(lambda row: mapping[row[map_label[0]]] if row[map_label[0]] in mapping else row[map_label[0]], axis=1)

    def translate(self, source:str, id:int, dest: str):
        # I could switch to a graph with nodes if this was ever more robust
        if (source, dest) in self.maps.keys():
            return self.maps[(source, dest)][id] if id in self.maps[(source, dest)] else id
        else:
            # print(f"Couldn't find {source} -> {dest} directly.")
            # This assumes there is only one path. No need to build graph
            _, next_dest = next(m for m in self.maps.keys() if m[0] == source)
            # print(f'Next hop {next_dest} -> {dest}')
            next_id = id if id not in self.maps[(source, next_dest)] else self.maps[(source, next_dest)][id]
            return self.translate(next_dest, next_id, dest)


In [ ]:
example = puzzle.examples[0]

# Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
# Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
# Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
# Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
print('Part A Test Cases:')
a = Almanac(example.input_data)
for (input, expected) in [(('seed', 79, 'soil'), 81), (('seed', 14, 'soil'), 14), (('seed', 55, 'soil'), 57), 
                          (('seed', 13, 'soil'), 13), (('seed', 79, 'fertilizer'), 81), (('seed', 79, 'water'), 81),
                          (('seed', 79, 'light'), 74), (('seed', 79, 'temperature'), 78), (('seed', 79, 'humidity'), 78),
                          (('seed', 79, 'location'), 82), (('seed', 14, 'location'), 43), (('seed', 55, 'location'), 86),
                          (('seed', 13, 'location'), 35)]:
    print(*input)
    test(a.translate, input, expected)

In [ ]:
a = Almanac(example.input_data)
print(f'Solution A: {min([a.translate('seed', s, 'location') for s in a.seeds])}')

In [ ]:
a = Almanac(puzzle.input_data)
print(f'Solution A: {min([a.translate('seed', s, 'location') for s in a.seeds])}')

Bummer! Memory error. I could prune down and eliminate anything that didn't have a seed, but I'm afraid that part B will look for a more general solution. Let's try to keep as little in memory as possible. I try a range or generator where needed first.

In [ ]:
import re
from collections import defaultdict


class LowMemoryAlmanac:

    def __init__(self, input_data:str):
        input_blocks = input_data.split('\n\n')
        
        self.seeds = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        
        maps = {}

        for block in input_blocks[1:]:
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()
            # print(f'Working on {map_label[0]} => {map_label[1]}')
            
            ranges = []
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                # TODO don't include anything that's not linked in an earlier map
                dest_start, source_start, range_len = m
                source_range = range(int(source_start), int(source_start) + int(range_len))
                dest_range = range(int(dest_start) , int(dest_start) + int(range_len))
                ranges.append((source_range, dest_range))
            maps[map_label] = ranges
            
        self.maps = maps
        
    def translate(self, source:str, id:int, dest: str):
        # I could switch to a graph with nodes if this was ever more robust
        if (source, dest) in self.maps.keys():
            # Direct translation
            ranges = next(((source_range, dest_range) for source_range, dest_range in self.maps[(source, dest)] if id in source_range), None)
            if not ranges:
                return id
            source_range, dest_range = ranges
            return next(dest for src, dest in zip(source_range, dest_range) if src == id)
        else:
            # print(f"Couldn't find {source} -> {dest} directly.")
            # This assumes there is only one path. No need to build graph
            _, next_dest = next(m for m in self.maps.keys() if m[0] == source)
            # print(f'Next hop {next_dest} -> {dest}')
            next_ranges = next(((source_range, next_range) for source_range, next_range in self.maps[(source, next_dest)] if id in source_range), None)
            next_id = id
            if next_ranges:
                source_range, dest_range = next_ranges
                next_id = next(dest for src, dest in zip(source_range, dest_range) if src == id)
            return self.translate(next_dest, next_id, dest)


In [ ]:
a = LowMemoryAlmanac(puzzle.examples[0].input_data)

# Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
# Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
# Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
# Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
print('Low Memory Part A Test Cases:')
for (input, expected) in [(('seed', 79, 'soil'), 81), (('seed', 14, 'soil'), 14), (('seed', 55, 'soil'), 57), 
                          (('seed', 13, 'soil'), 13), (('seed', 79, 'fertilizer'), 81), (('seed', 79, 'water'), 81),
                          (('seed', 79, 'light'), 74), (('seed', 79, 'temperature'), 78), (('seed', 79, 'humidity'), 78),
                          (('seed', 79, 'location'), 82), (('seed', 14, 'location'), 43), (('seed', 55, 'location'), 86),
                          (('seed', 13, 'location'), 35)]:
    print(*input)
    test(a.translate, input, expected)


In [ ]:
a = LowMemoryAlmanac(puzzle.input_data)

print(f'Solution A: {min([a.translate('seed', s, 'location') for s in a.seeds])}')

Wow! 31m 52.9s to finish. That's not ideal, but better than out of memory. I wonder if I could work backward from the lowest location to valid seed to be faster.

In [32]:
import re
from collections import defaultdict
from itertools import chain

class LowMemorySeedRangeAlmanac:

    def __init__(self, input_data:str):
        input_blocks = input_data.split('\n\n')
        
        seed_range_pairs = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        self.seeds = list(range(int(start), int(start) + int(len)) for start, len in zip(seed_range_pairs[::2], seed_range_pairs[1::2]))
        self.maps = {}

        for block in input_blocks[1:]:
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()
            # print(f'Working on {map_label[0]} => {map_label[1]}')
            
            ranges = []
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                # TODO don't include anything that's not linked in an earlier map
                dest_start, source_start, range_len = m
                source_range = range(int(source_start), int(source_start) + int(range_len))
                dest_range = range(int(dest_start) , int(dest_start) + int(range_len))
                ranges.append((source_range, dest_range))
            self.maps[map_label] = ranges
    def lowest_location_with_seed(self):
        pass

    def backtrack(self, dest: str, dest_id:int, source:str):
        if (source, dest) in self.maps.keys():
            if source != 'seed':
                raise Exception("Not implemented. This isn't a generic backtracking function.")
            # Direct translation
            ranges = next(((source_range, dest_range) for source_range, dest_range in self.maps[(source, dest)] if dest_id in dest_range), None)
            seed_id = None
            if not ranges:
                # We didn't find it in the table, so lets assume it came from a valid upstream table (i.e. it's missing in this source column, but because missing ids from a parent map to that same id lets treat it as if that happened) and pass the id from the next table up and assume it was a match
                seed_id = dest_id
            else:
                source_range, dest_range = ranges
                print(ranges)
                seed_id = next(src for src, dest in zip(source_range, dest_range) if dest == dest_id)

            # Special case. We translated back from say soil to seed, but we still need to check that the seed is valid.
            valid_seed_id = next((dest_id for r in self.seeds if dest_id in r), None)
            return valid_seed_id

        else:
            # print(f"Couldn't find {source} -> {dest} directly.")
            # This assumes there is only one path. No need to build graph
            _, next_dest = next(m for m in self.maps.keys() if m[0] == source)
            # print(f'Next hop {next_dest} -> {dest}')
            next_ranges = next(((source_range, next_range) for source_range, next_range in self.maps[(source, next_dest)] if id in source_range), None)
            next_id = id
            if next_ranges:
                source_range, dest_range = next_ranges
                next_id = next(dest for src, dest in zip(source_range, dest_range) if src == id)
            return self.translate(next_dest, next_id, dest)

a = LowMemorySeedRangeAlmanac(puzzle.examples[0].input_data)
print(a.maps)

# Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
# Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
# Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
# Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
print('Low Memory Part B Test Cases:')
for (input, expected) in [(('soil', 84, 'seed'), 82)]:
    print(*input)
    test(a.backtrack, input, expected)

print(a.seeds)

# print(f'Solution B Test: {min(a.backtrack('seed', s, 'location') for s in a.seeds)}')



{('seed', 'soil'): [(range(98, 100), range(50, 52)), (range(50, 98), range(52, 100))], ('soil', 'fertilizer'): [(range(15, 52), range(0, 37)), (range(52, 54), range(37, 39)), (range(0, 15), range(39, 54))], ('fertilizer', 'water'): [(range(53, 61), range(49, 57)), (range(11, 53), range(0, 42)), (range(0, 7), range(42, 49)), (range(7, 11), range(57, 61))], ('water', 'light'): [(range(18, 25), range(88, 95)), (range(25, 95), range(18, 88))], ('light', 'temperature'): [(range(77, 100), range(45, 68)), (range(45, 64), range(81, 100)), (range(64, 77), range(68, 81))], ('temperature', 'humidity'): [(range(69, 70), range(0, 1)), (range(0, 69), range(1, 70))], ('humidity', 'location'): [(range(56, 93), range(60, 97)), (range(93, 97), range(56, 60))]}
Low Memory Part B Test Cases:
soil 84 seed
(range(50, 98), range(52, 100))
	☐ - backtrack(('soil', 84, 'seed')) = 82 ≠ 84
[range(79, 93), range(55, 68)]


In [ ]:
print(puzzle.input_data[:100])

Hmm, looks like the seeds range is 7 digits. Not very like the example. I can see that the brute force isn't going to work.

According to https://en.wikipedia.org/wiki/Dynamic_programming, that still leaves backtracking and dynamic programming.

Let's check on the backtracking. Could I take the range of possibilites from the location, find the smallest, and work backward?

In [3]:
a = LowMemorySeedRangeAlmanac(puzzle.input_data)
print('How many location ids are there to check?')
print(sum([(location_range.stop - location_range.start - 1) for _, location_range in a.maps[('humidity', 'location')]]))
# print(sum([r.end - r.start for r in ]))

How many location ids are there to check?
4178076397


Well, `4178076397` seems like more than I'd be able to back track successfully.

In [7]:
a = LowMemorySeedRangeAlmanac(puzzle.input_data)
ranges = (location_range for _, location_range in a.maps[('humidity', 'location')])
print(len(list(ranges)))
ranges = (location_range for _, location_range in a.maps[('humidity', 'location')])
for r in ranges:
    print(len(list(r)))

47
54451455
28610653
103937617
21143365
24536388
98799841
251219053
24111266
126947592
9056683
18654108
392849235
93027519
39611681
46005642
19835810
11943322
109269701
16288912
75489801
56653992
14566173
61450145
51470789
32549578
11766607
44335967
193219887
133533079
66341838
209740195
59443964
129990669
520396542
93287087
97654013
41805547
98363636
110819236
15361857
21842201
173298030
27310773
33427002
48651863
31318817
233687313


Wait though. I could count the length of all the location generators in 6 min. I think I could tee off each generator (all 47) then pull the first number, check those backward until I find the best seed! I know I want the lowest location. If this was the cheapest path or something, I'd be in more trouble.

In [46]:
import heapq
import re

from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=5)


class BackwardLowMemorySeedRangeAlmanac:

    def __init__(self, input_data: str):
        input_blocks = input_data.split('\n\n')

        seed_range_pairs = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        self.seeds = list(range(int(start), int(start) + int(length)) for start, length in
                          zip(seed_range_pairs[::2], seed_range_pairs[1::2]))
        self.maps = {}

        for block in input_blocks[1:]:
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()
            # print(f'Working on {map_label[0]} => {map_label[1]}')

            ranges = []
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                # TODO don't include anything that's not linked in an earlier map
                dest_start, source_start, range_len = m
                source_range = range(int(source_start), int(source_start) + int(range_len))
                dest_range = range(int(dest_start), int(dest_start) + int(range_len))
                ranges.append((source_range, dest_range))
            self.maps[map_label] = ranges

    def lowest_location_with_seed(self):
        for low_loc in self._next_lowest_location():
            print(f'Looking from the lowest unchecked location {low_loc}')
            seed_id = self._dest_id_to_src_id('location', low_loc, 'seed')
            if seed_id:
                return low_loc

    def _next_lowest_location(self):
        loc_ranges = [iter(location_range) for _, location_range in self.maps[('humidity', 'location')]]
        h = []
        for loc_range in loc_ranges:
            loc_id = next(loc_range, None)
            if loc_id is not None:
                heapq.heappush(h, (loc_id, loc_range))
        while h[0]:
            loc_id, loc_range = heapq.heappop(h)
            next_loc_id = next(loc_range, None)
            if next_loc_id is not None:
                heapq.heappush(h, (next_loc_id, loc_range))
            yield loc_id
        return

    def _dest_id_to_src_id(self, dest: str, dest_id: int, src: str):
        # TODO Error handling
        src_id = None

        src_dest_map_key = next((m for m in self.maps.keys() if m[1] == dest), None)

        matching_range = next((r for r in self.maps[src_dest_map_key] if dest_id in r[1]), None)
        if not matching_range:
            # If it's not found, we have to assume that the id matched the table above.'
            print(f'Could not find {dest_id} in the range for {dest}. Assuming the source id is the same.')
            src_id = dest_id
        else:
            print(f'Found a matching {dest} range for {dest_id} in {matching_range}.')
            src_id = next(src for src, dest in zip(matching_range[0], matching_range[1]) if dest == dest_id)

        if src == src_dest_map_key[0]:
            # Base case. This map cleanly has the answer
            print(f'Base case since the map has the src {src_dest_map_key[0]} and the dest {src_dest_map_key[1]}')
            if src == 'seed':
                # If the src is seed, we need to do a quick check to confirm it's a real seed
                print(f'Seed case. Checking for existing seed {src_id}')
                seed_range = next((r for r in self.seeds if src_id in r), None)
                if not seed_range:
                    print(f'Not a seed! {src_id}')
                    return None
            return src_id
        else:
            # We've mapped an id, but we need to keep translating to reach the dest
            print(f"Couldn't map from {dest} to {src} directly. Will try to hop from {src_dest_map_key[0]} to {src}.")
            return self._dest_id_to_src_id(src_dest_map_key[0], src_id, src)



a = BackwardLowMemorySeedRangeAlmanac(puzzle.input_data)
example_almanac = BackwardLowMemorySeedRangeAlmanac(puzzle.examples[0].input_data)
print(f'Soluition B example: {example_almanac.lowest_location_with_seed()}')
print(f'Solution B: {a.lowest_location_with_seed()}')

TypeError: 'tuple' object is not an iterator

Two reasons that is a bad idea. Firstly, you can see from the example, that because of the way that unmatched
source ids carry through, that in the example case it couldn't jump backward from the lowest location (`46`)
because it wasn't in range.

Secondly, because of how inefficiently I'm mapping the arrays by zipping the complete ranges over and over,
it's never going to finish running in a reasonable amount of time.

In [ ]:
import itertools
import re

from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=5)


def test(method, input, expected):
    actual = method(*input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')


class LowMemorySeedRangeAlmanac:

    def __init__(self, input_data: str):
        input_blocks = input_data.split('\n\n')

        seed_range_pairs = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        self.seeds = list(range(int(start), int(start) + int(length)) for start, length in
                          zip(seed_range_pairs[::2], seed_range_pairs[1::2]))
        self.maps = {}

        for block in input_blocks[1:]:
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()
            # print(f'Working on {map_label[0]} => {map_label[1]}')

            ranges = []
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                # TODO don't include anything that's not linked in an earlier map
                dest_start, source_start, range_len = m
                source_range = range(int(source_start), int(source_start) + int(range_len))
                dest_range = range(int(dest_start), int(dest_start) + int(range_len))
                ranges.append((source_range, dest_range))
            self.maps[map_label] = ranges

    def translate(self, src: str, src_id: int, dest: str):
        return self._src_id_to_dest_id(src, src_id, dest)

    def _src_id_to_dest_id(self, src: str, src_id: int, dest: str):
        src_dest_map_key = next((m for m in self.maps.keys() if m[0] == src), None)
        matching_range = next((r for r in self.maps[src_dest_map_key] if src_id in r[0]), None)

        dest_id = None
        if not matching_range:
            # If it's not found, we have to assume that the id matched the table above.'
            # print(f'Could not find {src_id} in the range for {src}. Assuming the dest id is the same.')
            dest_id = src_id
        else:
            # print(f'Found a matching {src} range for {src_id} in {matching_range}.')
            indx = src_id - matching_range[0].start
            dest_id = matching_range[1][indx]

        if dest == src_dest_map_key[1]:
            # Base case. This map cleanly has the answer
            # print(
            #     f'Base case since the map has the src {src_dest_map_key[0]} and the dest {src_dest_map_key[1]}. Value is {dest_id}')
            return dest_id
        else:
            # print(f"Couldn't map from {src} to {dest} directly. Will try to hop from {src_dest_map_key[1]} to {dest}.")
            return self._src_id_to_dest_id(src_dest_map_key[1], dest_id, dest)

a = LowMemorySeedRangeAlmanac(puzzle.input_data)
seed_generator = ((i for i in r) for r in a.seeds)
print(f"Solution B: {min(a.translate('seed', s, 'location') for s in itertools.chain(*seed_generator))}")

Even with a more efficient algorithm, still too long! I need to work by range or I'll never finish.
I think I can translate the entire range for each range.

In [ ]:
import itertools
import re

from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=5)


def test(method, input, expected):
    actual = method(*input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')


class SeedLocationAlmanac:

    def __init__(self, input_data: str):
        input_blocks = input_data.split('\n\n')

        seed_range_pairs = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        self.seeds = list(range(int(start), int(start) + int(length)) for start, length in
                          zip(seed_range_pairs[::2], seed_range_pairs[1::2]))
        self.maps = {}

        for block in input_blocks[1:]:
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()
            # print(f'Working on {map_label[0]} => {map_label[1]}')

            ranges = []
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                # TODO don't include anything that's not linked in an earlier map
                dest_start, source_start, range_len = m
                source_range = range(int(source_start), int(source_start) + int(range_len))
                dest_range = range(int(dest_start), int(dest_start) + int(range_len))
                ranges.append((source_range, dest_range))
            self.maps[map_label] = ranges

    def _map_range(self, range_, ):
        pass
    def lowest_location(self):
        # These maps are ordered so they line up. I'm going to trust and not verify.

        # This is the working list of ranges that I'll translate through map by map
        # If the range isn't covered by the next map down, I'll have to split the range
        unmapped_src_ranges = self.seeds
        for _, ranges in self.maps.items():
            dest_ranges = []
            for map_src_range, map_dest_range in sorted(ranges, key=lambda x: x[1].start):
                for src_range in sorted(unmapped_src_ranges, key=lambda x: x.start):
                    if src_range.stop + 1 < map_src_range.start:
                        # The source is before the start so all ids pass through
                        dest_ranges.append(src_range)
                        unmapped_src_ranges.remove(src_range)
                    elif map_src_range.stop + 1 < src_range.start:
                        # The source is after this range so let's try again for the next one?
                        continue
                    elif map_src_range.start < src_range.start and src_range.stop < map_src_range.stop:
                        # The range fits totally in the source range
                        start = src_range.start - map_src_range.start
                        stop = map_src_range.stop - src_range.stop
                        dest_start = map_dest_range[map_dest_range.start + start]
                        dest_stop = map_dest_range[map_dest_range.start + stop]
                        # TODO really no reason to be working in ranges anymore. Start and stop are all I need.
                        dest_ranges.append(range(dest_start, dest_stop + 1))
                        unmapped_src_ranges.remove(src_range)
                    # TODO is there an issue with perfectly fitting ranges?
                    elif map_src_range.start < src_range.start:
                        # The range partially fits in the src range with the start passing through
                        dest_ranges.append(range(src_range.start, map_src_range.start))
                        start = map_src_range.start
                        stop = map_src_range.stop - src_range.end
                        dest_start = map_dest_range[map_dest_range.start + start]
                        dest_stop = map_dest_range[map_dest_range.start + stop]
                        dest_ranges.append(range(dest_start, dest_stop + 1))
                        unmapped_src_ranges.remove(src_range)
                    else:
                        # The range partially fits in the src range with the end passing through
                        dest_ranges.append(range(map_src_range.stop + 1, src_range.stop + 1))
                        start = src_range.start - map_src_range.start
                        stop = map_src_range.stop
                        dest_start = map_dest_range[map_dest_range.start + start]
                        dest_stop = map_dest_range[map_dest_range.start + stop]
                        dest_ranges.append(range(dest_start, dest_stop + 1))
                        unmapped_src_ranges.remove(src_range)
            unmapped_src_ranges = dest_ranges
            print(unmapped_src_ranges)
        # src_dest_map_key = next((m for m in self.maps.keys() if m[0] == src), None)
        # matching_range = next((r for r in self.maps[src_dest_map_key] if src_id in r[0]), None)
        #
        # dest_id = None
        # if not matching_range:
        #     # If it's not found, we have to assume that the id matched the table above.'
        #     # print(f'Could not find {src_id} in the range for {src}. Assuming the dest id is the same.')
        #     dest_id = src_id
        # else:
        #     # print(f'Found a matching {src} range for {src_id} in {matching_range}.')
        #     indx = src_id - matching_range[0].start
        #     dest_id = matching_range[1][indx]
        #
        # if dest == src_dest_map_key[1]:
        #     # Base case. This map cleanly has the answer
        #     # print(
        #     #     f'Base case since the map has the src {src_dest_map_key[0]} and the dest {src_dest_map_key[1]}. Value is {dest_id}')
        #     return dest_id
        # else:
        #     # print(f"Couldn't map from {src} to {dest} directly. Will try to hop from {src_dest_map_key[1]} to {dest}.")
        #     return self._src_id_to_dest_id(src_dest_map_key[1], dest_id, dest)


print(SeedLocationAlmanac(puzzle.input_data).lowest_location())

Ooof. I need to think this through and write tests

In [1]:
import logging
import re

from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=5)


def test(method, args, expected):
    actual = method(*args)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({args}) => {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({args}) => {expected} ≠ {actual}')


class SeedLocationAlmanac:

    def __init__(self, input_data: str):
        self.logger = logging.getLogger(__name__)

        input_blocks = input_data.split('\n\n')

        seed_range_pairs = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        self.seeds: [(range, range)] = list(range(int(start), int(start) + int(length)) for start, length in
                                            zip(seed_range_pairs[::2], seed_range_pairs[1::2]))
        self.seeds.sort(key=lambda r: r.start)
        self.maps = {}

        for block in input_blocks[1:]:
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()
            # print(f'Working on {map_label[0]} => {map_label[1]}')

            ranges = []
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                # TODO don't include anything that's not linked in an earlier map
                dest_start, source_start, range_len = m
                source_range = range(int(source_start), int(source_start) + int(range_len))
                dest_range = range(int(dest_start), int(dest_start) + int(range_len))
                ranges.append((source_range, dest_range))
            ranges.sort(key=lambda pair: pair[0].start)
            self.maps[map_label] = ranges

    def _has_overlap(self, a: range, b: range) -> bool:
        # TODO this assumes an order that a < b
        if a.stop <= b.start:
            return False
        if b.stop <= a.start:
            return False

        return True

    def _translate_range(self, src_range: range, translation_maps: list[tuple[range, range]]) -> list[range]:
        # Assume the translation_maps is sorted
        self.logger.debug(f'Mapping from src {src_range} using {translation_maps}')
        dest_ranges = []
        for map_src_range, map_dest_range in [m for m in translation_maps if self._has_overlap(src_range, m[0])]:
            self.logger.debug(f"\tMapping overlapping range {map_src_range} => {map_dest_range}")
            # Remember that stop isn't included in ranges

            #   |-src-|
            # |--map_src--|
            #
            # OR
            #
            # |----src----|
            # |--map_src--|
            # If everything left of the source is in the map, we can translate and be done.
            if map_src_range.start <= src_range.start and src_range.stop <= map_src_range.stop:
                self.logger.debug(f"\t{src_range} is inside {map_src_range} so we're done")
                # TODO here is where it's clear range objects are not helpful anymore. tuple[int,int] would work better
                dest_range = range(map_dest_range[src_range.start - map_src_range.start],
                                   map_dest_range[src_range.stop - map_src_range.start - 1] + 1)
                dest_ranges.append(dest_range)
                return dest_ranges

            # |---src---|
            #              |--map_src--|
            # TODO This case isn't possible since there isn't overlap with this range. This would have to come at the end of the for loop
            if src_range.stop <= map_src_range.start:
                self.logger.debug(f"\t{src_range} is before {map_src_range} so we're done")
                dest_ranges.append(src_range)
                self.logger.debug(f"Split into {dest_ranges}")
                return dest_ranges

            #        |------src------|
            #     |--map_src--|
            #
            if map_src_range.start <= src_range.start:
                # We know from the test above that the end of the src range is beyond the map range.
                self.logger.debug(f"\tThe src {src_range} starts (and ends) after the map range {map_src_range}")
                map_src_start_indx = src_range.start - map_src_range.start
                dest_ranges.append(range(map_dest_range[map_src_start_indx], map_dest_range[map_src_range.stop - 1 - map_src_range.start] + 1))
                src_range = range(map_src_range.stop, src_range.stop)
                continue
            # OR
            #
            # |------src--------|
            #     |--map_src--|
            #
            # OR
            #
            # |---src---|
            #    |-map_src-|
            # All or part of the src maps to the front part of this range. Could come from split from another range.
            if src_range.start < map_src_range.start:
                pre_range = range(src_range.start, map_src_range.start)
                self.logger.debug(f"\tThere is a beginning range {pre_range} not translated by {map_src_range}")
                dest_ranges.append(pre_range)
                src_range = range(pre_range.stop, src_range.stop)

                if src_range.stop <= map_src_range.stop:
                    self.logger.debug(f"\tThe rest of the src range {src_range} is in the map {map_src_range}")
                    dest_range = range(map_dest_range[0],
                                       map_dest_range[src_range.stop - map_src_range.start - 1] + 1)
                    dest_ranges.append(dest_range)
                    return dest_ranges

                self.logger.debug(f"\tThere is more src range {src_range} than mapped {map_src_range}")
                dest_range = range(map_dest_range.start,
                                   map_dest_range.stop)
                dest_ranges.append(dest_range)
                src_range = range(map_src_range.stop, src_range.stop)

        dest_ranges.append(src_range)
        return dest_ranges

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

almanac = SeedLocationAlmanac(puzzle.input_data)
# print(SeedLocationAlmanac(puzzle.input_data).lowest_location())

for (args, expected) in [
    # Single value ranges
    ((range(4, 5), [(range(5, 6), range(100, 101))]), [range(4, 5)]),  # src before
    ((range(7, 8), [(range(5, 6), range(100, 101))]), [range(7, 8)]),  # after before
    ((range(5, 6), [(range(5, 6), range(100, 101))]), [range(100, 101)]),  # overlap
    # Single value src, two value map
    ((range(4, 5), [(range(5, 7), range(100, 102))]), [range(4, 5)]),  # src before
    ((range(7, 8), [(range(5, 7), range(100, 103))]), [range(7, 8)]),  # after before
    ((range(5, 6), [(range(5, 7), range(100, 103))]), [range(100, 101)]),  # overlap first
    ((range(6, 7), [(range(5, 7), range(100, 103))]), [range(101, 102)]),  # overlap second
    # Two value range, single value map
    ((range(4, 6), [(range(6, 7), range(100, 101))]), [range(4, 6)]),  # src before
    ((range(6, 8), [(range(6, 7), range(100, 101))]), [range(100, 101), range(7, 8), ]),  # src overlap first
    ((range(5, 7), [(range(6, 7), range(100, 101))]), [range(5, 6), range(100, 101)]),  # src overlap second
    ((range(7, 9), [(range(6, 7), range(100, 101))]), [range(7, 9)]),  # src overlap after
    # Single value src, three value map
    ((range(4, 5), [(range(6, 9), range(100, 103))]), [range(4, 5)]),  # src before
    ((range(6, 7), [(range(6, 9), range(100, 103))]), [range(100, 101)]),  # src overlap first
    ((range(7, 8), [(range(6, 9), range(100, 103))]), [range(101, 102)]),  # src overlap second
    ((range(8, 9), [(range(6, 9), range(100, 103))]), [range(102, 103)]),  # src overlap third
    ((range(9, 10), [(range(6, 9), range(100, 103))]), [range(9, 10)]),  # src overlap after
    #
    ((range(5, 10), [(range(5, 10), range(10, 15))]), [range(10, 15)]),  # Overlapped src and map
    ((range(7, 9), [(range(5, 10), range(10, 15))]), [range(12, 14)]),  # Src within map
    ((range(6, 9), [(range(5, 10), range(10, 15))]), [range(11, 14)]),  # Src within map
    ((range(3, 9), [(range(5, 10), range(10, 15))]), [range(3, 5), range(10, 14)]),  # Src unmapped start
    ((range(3, 13), [(range(5, 10), range(10, 15))]), [range(3, 5), range(10, 15), range(10, 13)]),  # Map within src
    ((range(9, 11), [(range(5, 10), range(10, 15))]), [range(14, 15), range(10, 11)]),  # Map within src
]:
    test(almanac._translate_range, args, expected)


almanac = SeedLocationAlmanac(puzzle.input_data)
ranges = almanac.seeds
for _, maps in almanac.maps.items():
    dest_ranges = []
    for r in ranges:
        dest_ranges = dest_ranges + almanac._translate_range(r, maps)
    ranges = sorted(dest_ranges, key=lambda r: r.start)
print(f"Solution B: {ranges[0]}")

DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_05_input.txt
DEBUG:__main__:Mapping from src range(4, 5) using [(range(5, 6), range(100, 101))]
DEBUG:__main__:Mapping from src range(7, 8) using [(range(5, 6), range(100, 101))]
DEBUG:__main__:Mapping from src range(5, 6) using [(range(5, 6), range(100, 101))]
DEBUG:__main__:	Mapping overlapping range range(5, 6) => range(100, 101)
DEBUG:__main__:	range(5, 6) is inside range(5, 6) so we're done
DEBUG:__main__:Mapping from src range(4, 5) using [(range(5, 7), range(100, 102))]
DEBUG:__main__:Mapping from src range(7, 8) using [(range(5, 7), range(100, 103))]
DEBUG:__main__:Mapping from src range(5, 6) using [(range(5, 7), range(100, 103))]
DEBUG:__main__:	Mapping overlapping range range(5, 7) => range(100, 103)
DEBUG:__main__:	range(5, 6) is inside range(5, 7) so we're done
DEBUG:__main__:Mapping from src range(6, 7) using [(range(5, 7), range(100, 103))]
DEBUG:__main__:	Mapping overlapping 

	☑ - _translate_range((range(4, 5), [(range(5, 6), range(100, 101))])) => [range(4, 5)] = [range(4, 5)]
	☑ - _translate_range((range(7, 8), [(range(5, 6), range(100, 101))])) => [range(7, 8)] = [range(7, 8)]
	☑ - _translate_range((range(5, 6), [(range(5, 6), range(100, 101))])) => [range(100, 101)] = [range(100, 101)]
	☑ - _translate_range((range(4, 5), [(range(5, 7), range(100, 102))])) => [range(4, 5)] = [range(4, 5)]
	☑ - _translate_range((range(7, 8), [(range(5, 7), range(100, 103))])) => [range(7, 8)] = [range(7, 8)]
	☑ - _translate_range((range(5, 6), [(range(5, 7), range(100, 103))])) => [range(100, 101)] = [range(100, 101)]
	☑ - _translate_range((range(6, 7), [(range(5, 7), range(100, 103))])) => [range(101, 102)] = [range(101, 102)]
	☑ - _translate_range((range(4, 6), [(range(6, 7), range(100, 101))])) => [range(4, 6)] = [range(4, 6)]
	☑ - _translate_range((range(6, 8), [(range(6, 7), range(100, 101))])) => [range(100, 101), range(7, 8)] = [range(100, 101), range(7, 8)]
	☑ - _

DEBUG:__main__:Mapping from src range(1890784901, 1906717319) using [(range(192320896, 220931957), range(2212995455, 2241606516)), (range(220931957, 431490014), range(1203574278, 1414132335)), (range(431490014, 498932829), range(1744764149, 1812206964)), (range(498932829, 598391916), range(417196290, 516655377)), (range(598391916, 598680988), range(2241606516, 2241895588)), (range(598680988, 744338927), range(525827016, 671484955)), (range(744338927, 966203327), range(1991131055, 2212995455)), (range(966203327, 1135735535), range(2241895588, 2411427796)), (range(1135735535, 1144907174), range(516655377, 525827016)), (range(1144907174, 1676996497), range(671484955, 1203574278)), (range(1676996497, 1775426812), range(1892700740, 1991131055)), (range(1775426812, 1960778188), range(1559412773, 1744764149)), (range(1960778188, 2086495209), range(1414132335, 1539849356)), (range(2086495209, 2106058626), range(1539849356, 1559412773)), (range(2106058626, 2186552402), range(1812206964, 1892700

Solution B: range(2008785, 13286162)
